# Comparador de Constitucion Chile - España

Usaremos LangChain para comparar la constitución actual chilena (del 80') con la española.

In [1]:
from functools import partial

from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import ConversationalRetrievalChain
# from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredPDFLoader

In [4]:
# from langchain.callbacks.wandb_tracer import 
# WandbTracer.watch_all({"project": "es-const-llm"})

## Setup

Cargamos los documentos en PDF directamente

In [5]:
dl = DirectoryLoader(path="./data", 
                     glob="Constitucion*")

In [4]:
docs = dl.load()

detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.
detectron2 is not installed. Cannot use the hi_res partitioning strategy. Falling back to partitioning with the fast strategy.


Chile

In [6]:
docs[0].page_content[:400]

'CONSTITUCIÓN POLÍTICA DE LA REPÚBLICA\n\nTexto actualizado a octubre  de 2010\n\n2\n\n3\n\nCONSTITUCIÓN POLÍTICA DE LA REPÚBLICA DE CHILE\n\nINDICE\n\nCAPÍTULOS\n\nCAPITULO I  BASES DE LA INSTITUCIONALIDAD\n\nCAPITULO II  NACIONALIDAD Y CIUDADANIA\n\nCAPITULO III  DE LOS DERECHOS Y DEBERES\n\nCONSTITUCIONALES\n\nCAPITULO IV  GOBIERNO\n\nPresidente de la República\n\nMinistros de Estado\n\nArts.   1 al 9\n\nArts.  10 al 18\n\nArt'

España

In [7]:
docs[1].page_content[:400]

'CONSTITUCIÓN\n\nESPAÑOLA\n\nAprobada por Las Cortes en sesiones plenarias del\n\nCongreso de los Diputados y del Senado celebradas\n\nel 31 de octubre de 1978\n\nRatificada por el pueblo español en referéndum de\n\n6 de diciembre de 1978\n\nSancionada por S. M. el Rey ante Las Cortes\n\nel 27 de diciembre de 1978\n\n© Agencia Estatal Boletín Oficial del Estado\n\nNIPO: 007-14-093-3\n\nCatálogo de Publicaciones de la Ad'

Hacemos split en pedazos para mandarlo al embedding...

In [8]:
text_splitter = CharacterTextSplitter(chunk_size=1024, 
                                      chunk_overlap=200)
splitted_docs = text_splitter.split_documents(docs)

let's embed using OpenAI Ada02

In [7]:
embeddings = OpenAIEmbeddings(chunk_size=1024)

In [11]:
# db = Chroma.from_documents(texts, embeddings)
db = FAISS.from_documents(splitted_docs, embeddings)

we can save a load back...

In [13]:
db.save_local("./data/constituciones.faiss")

In [10]:
# db = FAISS.load_local("./data/constituciones.faiss", embeddings)

We store everything in the vector database `db`

In [11]:
query = "Cuantos años dura el mandato del presidente"

In [12]:
retriever = db.as_retriever(search_kwargs=dict(k=5))

In [13]:
db.similarity_search(query, k=5)

[Document(page_content='el décimo día después de su proclamación.\n\nEl Presidente elegido conforme a alguno de los incisos precedentes durará en el\n\ncargo hasta completar el período que restaba a quien se reemplace y no podrá postular\n\ncomo candidato a la elección presidencial siguiente.86\n\n83 Frase intercalada por el artículo único, letra c) de la ley de Reforma Constitucional N° 20.354.\n\n84 Inciso modificado por el artículo único, Nº 12 de la ley de Reforma Constitucional Nº 18.825.\n\n85 Inciso modificado por el artículo único, letra d),  de la ley de Reforma Constitucional N° 20.354.\n\n86  Artículo  sustituido  por  el  artículo  1º,  Nº  16  de  la  ley  de  Reforma  Constitucional  Nº  20.050.  Anteriormente  había  sido\n\nmodificado por el artículo único, Nº 13 de la ley de Reforma Constitucional Nº 18.825.\n\n27\n\nArtículo  30.-  El  Presidente  cesará  en  su  cargo  el  mismo  día  en  que  se\n\ncomplete su período y le sucederá el recientemente elegido.', metada

Creamos un chat para recuperar info de la base de datos...

In [14]:
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(), 
                                           retriever, 
                                           return_source_documents=True)

In [15]:
chat_history = []
result = qa({"question": query, "chat_history": chat_history})

In [17]:
result['answer']

'El Presidente de la República durará en el ejercicio de sus funciones por el término de cuatro años y no podrá ser reelegido para el período siguiente. (Artículo 25)'

In [16]:
len(result["source_documents"])

5

In [66]:
result["source_documents"]

[Document(page_content='orden  público  en  el  interior  y  la  seguridad  externa  de  la  República,  de  acuerdo  con  la\n\nConstitución y las leyes.\n\nEl 21 de mayo de cada año, el Presidente de la República dará cuenta al país del\n\nestado administrativo y político de la Nación ante el Congreso Pleno.71\n\nArtículo  25.  Para  ser  elegido  Presidente  de  la  República  se  requiere  tener  la\n\nnacionalidad  chilena  de  acuerdo  a  lo  dispuesto  en  los  números  1º  ó  2º  del  artículo  10;\n\ntener  cumplidos  treinta  y  cinco  años  de  edad  y  poseer  las  demás  calidades  necesarias\n\npara ser ciudadano con derecho a sufragio.72\n\nEl  Presidente  de  la  República  durará  en  el  ejercicio  de  sus  funciones  por  el\n\ntérmino de cuatro años y no podrá ser reelegido para el período siguiente.73\n\nEl  Presidente  de  la  República  no  podrá  salir  del  territorio  nacional  por  más  de\n\ntreinta días ni en los últimos noventa días de su período, sin acue

## Comparemos las constituciones...

In [67]:
chat_history = []
query = "Cual pais tiene un sistema parlamentario?"
result = qa({"question": query, "chat_history": chat_history})

In [68]:
result['answer']

'Varios países tienen un sistema parlamentario, en el que el poder legislativo está compuesto por una cámara o parlamento, que es elegido por el pueblo y tiene la función de hacer leyes y controlar al gobierno. Algunos ejemplos de países con sistema parlamentario son el Reino Unido, Australia, Canadá, India, Japón y España.'

In [69]:
result['source_documents'][0]

Document(page_content='los demás en proporción a la población.\n\n3.    La elección se verificará en cada circunscripción atendiendo a\n\ncriterios de representación proporcional.\n\n4.    El Congreso es elegido por cuatro años. El mandato de los\n\nDiputados termina cuatro años después de su elección o el día de la\n\ndisolución de la Cámara.\n\n5.    Son electores y elegibles todos los españoles que estén en\n\npleno uso de sus derechos políticos.\n\nLa ley reconocerá y el Estado facilitará el ejercicio del derecho de\n\nsufragio a los españoles que se encuentren fuera del territorio de\n\nEspaña.\n\nEl mandato\n\nparlamentario\n\nEl Congreso de\n\nlos Diputados\n\nSistema\n\nelectoral\n\nCuatro años de\n\nlegislatura\n\n23\n\n6.    Las elecciones tendrán lugar entre los treinta días y sesenta\n\ndías desde la terminación del mandato. El Congreso electo deberá\n\nser convocado dentro de los veinticinco días siguientes a la celebra-\n\nción de las elecciones.\n\nArtículo 69.\n\n1.    

In [15]:
chat_history = [(query, result["answer"])]
query = "Y chile?"
result = qa({"question": query, "chat_history": chat_history})

In [16]:
result['answer']

'Chile tiene un sistema presidencialista, en el cual el Presidente de la República es el jefe de Estado y de gobierno. Si bien el país cuenta con un Congreso Nacional, este no tiene un rol predominante en la definición y conducción de la política gubernamental.'

In [17]:
result['source_documents'][0]

Document(page_content='2 Inciso modificado por el artículo único, N° 1 de la ley de Reforma Constitucional N° 19.611.\n\nArtículo\n\n2°\n\nSon\n\nemblemas  nacionales  la  bandera  nacional,  el  escudo\n\nde armas de la República y el himno nacional.\n\n6\n\nArtículo 3º El Estado de Chile es unitario.\n\nLa  administración  del  Estado será  funcional  y  territorialmente descentralizada, o\n\ndesconcentrada en su caso, de conformidad a la ley.\n\nLos  órganos  del  Estado  promoverán  el  fortalecimiento  de  la  regionalización  del\n\npaís  y  el  desarrollo  equitativo  y  solidario  entre  las  regiones,  provincias  y  comunas  del\n\nterritorio nacional.3\n\nArtículo 4° Chile es una república democrática.\n\nArtículo  5°  La  soberanía  reside  esencialmente  en  la  Nación.  Su  ejercicio  se\n\nrealiza por el pueblo a través del plebiscito y de elecciones periódicas y, también, por las\n\nautoridades que esta Constitución establece. Ningún sector del pueblo ni individuo algun

In [18]:
chat_history = [(query, result["answer"])]
query = "Puedes explicarme la diferencia entre los dos?"
result = qa({"question": query, "chat_history": chat_history})

In [19]:
result['answer']

'En un sistema presidencialista, el presidente es el jefe de estado y de gobierno y tiene poderes ejecutivos y legislativos para tomar decisiones sin tener que depender del Congreso para aprobar medidas. Mientras que en un sistema en el que el Congreso tiene un rol predominante, el poder ejecutivo es ejercido por un primer ministro o un jefe de gobierno y depende de la aprobación del Congreso para llevar a cabo sus medidas. En este sistema, el Congreso tiene un papel activo en la definición y conducción de la política gubernamental y el poder del jefe de gobierno está más limitado que en un sistema presidencialista.'

In [20]:
chat_history = [(query, result["answer"])]
query = "En españa el rey tiene algun poder?"
result = qa({"question": query, "chat_history": chat_history})

In [21]:
result['answer']

'En España, el rey tiene algunos poderes ejecutivos, como el derecho de sancionar y promulgar las leyes, convocar y disolver las Cortes Generales y nombrar y separar a los miembros del Gobierno a propuesta de su Presidente, entre otros. También tiene algunos poderes simbólicos, como prestar juramento de fidelidad al país y recibir acreditaciones de embajadores extranjeros.\n\nComparado con sistemas presidencialistas, como en Estados Unidos, el presidente tiene un poder ejecutivo mucho mayor y su elección es directa. En el caso de España, el Rey tiene un poder simbólico muy importante y no es elegido directamente, sino que su sucesión se define por la ley. \n\nEn sistemas parlamentarios donde el Congreso tiene un rol predominante, como en el Reino Unido, el monarca tiene aún menos poderes ejecutivos y su principal rol es representativo, como en España. En ambos sistemas, el líder del gobierno es elegido por el parlamento o por los partidos políticos, aunque en España esto se hace a part

In [22]:
result["source_documents"][0]

Document(page_content='el nombramiento del Presidente del Gobierno, y la disolución prevista\n\nen el artículo 99, serán refrendados por el Presidente del Congreso.\n\n2.    De los actos del Rey serán responsables las personas que los\n\n22\n\nrefrenden.\n\nArtículo 65.\n\n1.    El Rey recibe de los Presupuestos del Estado una cantidad glo-\n\nbal para el sostenimiento de su Familia y Casa, y distribuye libremen-\n\nte la misma.\n\n2.    El Rey nombra y releva libremente a los miembros civiles y\n\nmilitares de su Casa.\n\nLa Casa del\n\nRey\n\nTÍTULO III\n\nDe las Cortes Generales\n\nCAPÍTULO PRIMERO\n\nDe las Cámaras\n\nArtículo 66.\n\n1.    Las Cortes Generales representan al pueblo español y están\n\nformadas por el Congreso de los Diputados y el Senado.\n\n2.    Las Cortes Generales ejercen la potestad legislativa del Esta-\n\ndo, aprueban sus Presupuestos, controlan la acción del Gobierno y\n\ntienen las demás competencias que les atribuya la Constitución.\n\n3.    Las Cortes Gen